In [1]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"sibtainwali","key":"dab2f11f7979d1efda7e5b520ec68dc2"}'}

In [3]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [4]:
!kaggle datasets list -s "loan prediction"


ref                                                    title                                               size  lastUpdated                 downloadCount  voteCount  usabilityRating  
-----------------------------------------------------  --------------------------------------------  ----------  --------------------------  -------------  ---------  ---------------  
kmldas/loan-default-prediction                         Loan Default Prediction                           111675  2021-04-06 15:03:41.200000          11129        107  1.0              
bhavikjikadara/loan-status-prediction                  Loan Status Prediction                              5336  2024-02-02 13:27:20.870000          12377        103  1.0              
architsharma01/loan-approval-prediction-dataset        Loan-Approval-Prediction-Dataset                   82535  2023-07-16 16:31:20.923000          30402        195  0.9411765        
nikhil1e9/loan-default                                 Loan Default Predict

In [5]:
!kaggle datasets download -d altruistdelhite04/loan-prediction-problem-dataset
!unzip -o loan-prediction-problem-dataset.zip


Dataset URL: https://www.kaggle.com/datasets/altruistdelhite04/loan-prediction-problem-dataset
License(s): unknown
  0% 0.00/12.6k [00:00<?, ?B/s]
100% 12.6k/12.6k [00:00<00:00, 28.7MB/s]
Archive:  loan-prediction-problem-dataset.zip
  inflating: test_Y3wMUE5_7gLdaTN.csv  
  inflating: train_u6lujuX_CVtuZ9i.csv  


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import joblib


df = pd.read_csv("train_u6lujuX_CVtuZ9i.csv")


imputer = SimpleImputer(strategy='most_frequent')
df_imputed = pd.DataFrame(imputer.fit_transform(df), columns=df.columns)


encoders = {}
for col in df_imputed.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    df_imputed[col] = le.fit_transform(df_imputed[col])
    encoders[col] = le  


X = df_imputed.drop(["Loan_ID", "Loan_Status"], axis=1)
y = df_imputed["Loan_Status"]


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)


y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))


joblib.dump(model, "loan_model.pkl")
joblib.dump(list(X.columns), "loan_model_features.pkl")
joblib.dump(encoders, "loan_model_encoders.pkl")


Accuracy: 0.7560975609756098

Classification Report:
               precision    recall  f1-score   support

           0       0.78      0.42      0.55        43
           1       0.75      0.94      0.83        80

    accuracy                           0.76       123
   macro avg       0.77      0.68      0.69       123
weighted avg       0.76      0.76      0.73       123


Confusion Matrix:
 [[18 25]
 [ 5 75]]


['loan_model_encoders.pkl']

In [ ]:
%%writefile app.py
import streamlit as st
import pandas as pd
import joblib


model = joblib.load("loan_model.pkl")
feature_names = joblib.load("loan_model_features.pkl")
encoders = joblib.load("loan_model_encoders.pkl")


def encode_input(data_dict):
    encoded_dict = {}
    for col in data_dict:
        if col in encoders:
            encoded_dict[col] = encoders[col].transform([data_dict[col]])[0]
        else:
            encoded_dict[col] = data_dict[col]
    return pd.DataFrame([encoded_dict], columns=feature_names)


st.title("🏦 Loan Approval Prediction System")


st.sidebar.header("Quick Test Cases")
if st.sidebar.button("✅ Likely Approved"):
    test_case = {
        "Gender": "Male",
        "Married": "Yes",
        "Dependents": "0",
        "Education": "Graduate",
        "Self_Employed": "No",
        "ApplicantIncome": 6000,
        "CoapplicantIncome": 2000,
        "LoanAmount": 120,
        "Loan_Amount_Term": 360,
        "Credit_History": 1.0,
        "Property_Area": "Urban"
    }
    df_input = encode_input(test_case)
    prediction = model.predict(df_input)[0]
    st.success(f"Prediction: {'✅ Approved' if prediction == 1 else '❌ Rejected'}")

if st.sidebar.button("❌ Likely Rejected"):
    test_case = {
        "Gender": "Female",
        "Married": "No",
        "Dependents": "2",
        "Education": "Not Graduate",
        "Self_Employed": "Yes",
        "ApplicantIncome": 1500,
        "CoapplicantIncome": 0,
        "LoanAmount": 200,
        "Loan_Amount_Term": 360,
        "Credit_History": 0.0,
        "Property_Area": "Rural"
    }
    df_input = encode_input(test_case)
    prediction = model.predict(df_input)[0]
    st.success(f"Prediction: {'✅ Approved' if prediction == 1 else '❌ Rejected'}")

if st.sidebar.button("⚖️ Borderline Case"):
    test_case = {
        "Gender": "Male",
        "Married": "Yes",
        "Dependents": "1",
        "Education": "Graduate",
        "Self_Employed": "No",
        "ApplicantIncome": 3500,
        "CoapplicantIncome": 1500,
        "LoanAmount": 150,
        "Loan_Amount_Term": 360,
        "Credit_History": 1.0,
        "Property_Area": "Semiurban"
    }
    df_input = encode_input(test_case)
    prediction = model.predict(df_input)[0]
    st.success(f"Prediction: {'✅ Approved' if prediction == 1 else '❌ Rejected'}")


st.sidebar.header("Custom Applicant Details")
Gender = st.sidebar.selectbox("Gender", encoders["Gender"].classes_)
Married = st.sidebar.selectbox("Married", encoders["Married"].classes_)
Dependents = st.sidebar.selectbox("Dependents", encoders["Dependents"].classes_)
Education = st.sidebar.selectbox("Education", encoders["Education"].classes_)
Self_Employed = st.sidebar.selectbox("Self_Employed", encoders["Self_Employed"].classes_)
ApplicantIncome = st.sidebar.number_input("Applicant Income", 0)
CoapplicantIncome = st.sidebar.number_input("Coapplicant Income", 0)
LoanAmount = st.sidebar.number_input("Loan Amount (in $)", 0)
Loan_Amount_Term = st.sidebar.number_input("Loan Amount Term (in days)", 0)
Credit_History = st.sidebar.selectbox("Credit History", encoders["Credit_History"].classes_)
Property_Area = st.sidebar.selectbox("Property_Area", encoders["Property_Area"].classes_)

if st.sidebar.button("Predict Custom"):
    custom_case = {
        "Gender": Gender,
        "Married": Married,
        "Dependents": Dependents,
        "Education": Education,
        "Self_Employed": Self_Employed,
        "ApplicantIncome": ApplicantIncome,
        "CoapplicantIncome": CoapplicantIncome,
        "LoanAmount": LoanAmount,
        "Loan_Amount_Term": Loan_Amount_Term,
        "Credit_History": Credit_History,
        "Property_Area": Property_Area
    }
    df_input = encode_input(custom_case)
    prediction = model.predict(df_input)[0]
    st.success(f"Prediction: {'✅ Approved' if prediction == 1 else '❌ Rejected'}")


Overwriting app.py


In [20]:
!pip install -q streamlit
!npm install -g localtunnel
!streamlit run app.py & npx localtunnel --port 8501


⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹
changed 22 packages in 2s
⠹
⠹3 packages are looking for funding
⠹  run `npm fund` for details
⠹⠙

⠹⠸⠼
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.133.144.19:8501

⠴⠦⠧⠇⠏⠋⠙your url is: https://full-peas-carry.loca.lt
  Stopping...
^C


In [10]:
!curl https://loca.lt/mytunnelpassword


34.133.144.19